In [2]:
import aux_document_retrieval_bm25 as aux_bm25
import aux_document_retrieval_vsm as aux_vsm
import aux_semantic_search as aux_semantics
import aux_retrieval_evaluation as aux_retrieval
import aux_document_retrieval_hybrid as aux_hybrid

import pandas as pd
from pathlib import Path
import logging
import nltk
from dataclasses import dataclass
import json
import importlib


importlib.reload(aux_vsm)
importlib.reload(aux_bm25)
importlib.reload(aux_semantics)
importlib.reload(aux_retrieval)

nltk.download('punkt_tab')


2025-05-26 11:18:57,319 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.
[nltk_data] Downloading package punkt_tab to /home/pablo/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
## Basic enviroment setup

paths = {
    'word2vec': Path("../02-data/03-VSM/01-Word2Vec/word2vec-google-news-300.bin"),
    'idf_cache': Path("../02-data/03-VSM/idf_cache_path.pkl"),
    'word2vec_vsm_multivector': Path("../02-data/03-VSM/01-Word2Vec/word2vec-4-50-4-150-1.pkl"),
    'word2vec_vsm_singlevector': Path("../02-data/03-VSM/01-Word2Vec/word2vec-4-50-4-150-0.pkl"),
    'file': Path("../02-data/00-testing/batteries-non-rechargable-primary/1cr2/1cr2.txt"),
    'output_path': Path("../02-data/00-testing/batteries-non-rechargable-primary/1cr2/sentence_expansions.txt"),
    'pdf_folder': Path("../02-data/00-testing/"),
    'retriever': Path("../02-data/05-Retrieval/corpus_bm25")
}

@dataclass
class DocumentSection:
    doc: str
    label: str
    query: str
    
with open("../02-data/06-Evaluation/document_queries.json") as f:
    raw = json.load(f)

query_list_documents = [DocumentSection(**entry) for entry in raw]

In [ ]:
## VSM evaluation

summary_vsm_df, results_map = aux_retrieval.evaluate_vsm_queries(
    paths,
    query_list_documents,
    top_k=20,
    use_multivector=True,
    use_expansion=True
)

display(summary_vsm_df)
display(results_map['1cr2'][:5])

In [87]:
## BM25 evaluation

logging.disable(logging.CRITICAL) ##im just tired ok? this is it

summary_bm25_df, results_bm25 = aux_retrieval.evaluate_queries(
    paths=paths,
    documents=query_list_documents,
    top_k=20,
    method="bm25"
)
logging.disable(logging.NOTSET)

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

...

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
display(summary_bm25_df)
display(results_bm25['1cr2'][:5])

In [ ]:
summary_hybrid_df,results_hybrid = aux_retrieval.evaluate_hybrid_queries(
    paths=paths,
    documents=query_list_documents,
    top_k=200,
    weight_vsm=0.6,
    weight_bm25=0.4,
    norm_vsm='minmax',
    norm_bm25='minmax',
    use_expansion=True,
    use_multivector=False,
)

In [ ]:
summary_hybrid_df

In [ ]:
##make graphs pretty and whatnot

import matplotlib.pyplot as plt

# 1) Descriptive statistics
stats = summary_df[[
    'label_count_top5',
    'label_count_top10',
    'label_count_top20',
    'matched_rank',
    'matched_score'
]].describe()
print("Descriptive statistics:\n", stats)


# 2) Histograms for each label_count_*
for col in ['label_count_top5', 'label_count_top10', 'label_count_top20']:
    plt.figure()
    plt.hist(summary_df[col].dropna(), bins=range(0, summary_df[col].max() + 2))
    plt.title(f'Histogram of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()


# 3) Scatter plot: matched_rank vs. label_count_top20
#    (only for rows where matched_rank is not null)
valid = summary_df.dropna(subset=['matched_rank'])
plt.figure()
plt.scatter(valid['matched_rank'], valid['label_count_top20'])
plt.title('Matched Rank vs. Label Count Top20')
plt.xlabel('Matched Rank')
plt.ylabel('Label Count Top20')
plt.show()

In [85]:
## run single query for word2vec
top_k = 100
use_expansion = True

resources = aux_vsm.load_word2vec_resources(
    paths,
    use_multivector=False
)
query_runner = lambda q: aux_vsm.run_word2vec_query_preloaded(
    resources=resources,
    query=q,
    top_k=top_k,
    use_expansion=use_expansion
)


raw_result = query_runner(query_list_documents[0].query)
df = pd.DataFrame(data = raw_result['results'])
df


,rank,doc_id,score,parent,grandparent
0,1,SR41-392-384-DATASHEET,0.728945,SR41-392-384-DATASHEET.txt,SR41-392-384-DATASHEET
1,2,392-384z,0.726830,392-384z.txt,392-384z
2,3,GT-0905A-19-Feb-2019,0.723888,GT-0905A-19-Feb-2019.txt,GT-0905A-19-Feb-2019
3,4,maxell-alkaline-battery-002-2014,0.718862,maxell-alkaline-battery-002-2014.txt,maxell-alkaline-battery-002-2014
4,5,MU064602-1,0.716878,MU064602-1.txt,MU064602-1
...,...,...,...,...,...
95,96,cms-16093-078x-67,0.685307,cms-16093-078x-67.txt,cms-16093-078x-67
96,97,cem-1212s,0.685244,cem-1212s.txt,cem-1212s
97,98,cms-160925-18sp-x8,0.685211,cms-160925-18sp-x8.txt,cms-160925-18sp-x8
98,99,cds-15158-smt,0.685202,cds-15158-smt.txt,cds-15158-smt


In [ ]:
logging.disable(logging.CRITICAL) ##im just tired ok? this is it  --- look more into it, does not seem right

weight_results = aux_retrieval.optimize_hybrid_weights(aux_retrieval, paths, query_list_documents)
display(weight_results)

In [84]:
display(weight_results)

,weight_vsm,weight_bm25,median_rank,count_lt10
0,0.9,0.1,97.5,1
1,1.0,0.0,97.5,1
2,0.8,0.2,102.5,1
3,0.7,0.3,121.5,1
4,0.6,0.4,142.5,1
5,0.5,0.5,164.0,1
6,0.4,0.6,188.5,0
7,0.3,0.7,215.5,0
8,0.2,0.8,239.0,0
9,0.1,0.9,264.0,0


In [93]:
display(results_bm25['1cr2'])
display(df)

,doc_id,doc_name,label,score
rank,,,,
1,37,123,batteries-non-rechargable-primary,5.875960
2,38,1cr2,batteries-non-rechargable-primary,5.740847
3,509,cms-402008-18sp,speakers,4.363099
4,105,maxell-alkaline-battery-002-2014,batteries-non-rechargable-primary,4.257775
5,60,alkaline_appman,batteries-non-rechargable-primary,4.058401
6,109,MN1604_US_CT1,batteries-non-rechargable-primary,4.015090
7,112,Rayovac-Size-675-DS,batteries-non-rechargable-primary,3.697664
8,101,LR927SPEC,batteries-non-rechargable-primary,3.651839
9,100,LR920SPEC,batteries-non-rechargable-primary,3.440841


,rank,doc_id,score,parent,grandparent
0,1,SR41-392-384-DATASHEET,0.728945,SR41-392-384-DATASHEET.txt,SR41-392-384-DATASHEET
1,2,392-384z,0.726830,392-384z.txt,392-384z
2,3,GT-0905A-19-Feb-2019,0.723888,GT-0905A-19-Feb-2019.txt,GT-0905A-19-Feb-2019
3,4,maxell-alkaline-battery-002-2014,0.718862,maxell-alkaline-battery-002-2014.txt,maxell-alkaline-battery-002-2014
4,5,MU064602-1,0.716878,MU064602-1.txt,MU064602-1
...,...,...,...,...,...
95,96,cms-16093-078x-67,0.685307,cms-16093-078x-67.txt,cms-16093-078x-67
96,97,cem-1212s,0.685244,cem-1212s.txt,cem-1212s
97,98,cms-160925-18sp-x8,0.685211,cms-160925-18sp-x8.txt,cms-160925-18sp-x8
98,99,cds-15158-smt,0.685202,cds-15158-smt.txt,cds-15158-smt


In [9]:
logging.basicConfig(
    level=logging.ERROR,
    format="%(asctime)s %(name)s [%(levelname)s] %(message)s",
    force=True
)

# Now all loggers (including your module’s) will use this new config.
logger = logging.getLogger(__name__)
logger.error("This is a test at error level")

importlib.reload(aux_vsm)
top_k = 10
use_expansion = True
use_multivector = True

resources = aux_vsm.load_word2vec_resources(paths, use_multivector=use_multivector)
results = aux_vsm.run_word2vec_query_preloaded(resources, query_list_documents[0].query, top_k=top_k,use_expansion=use_expansion)

2025-05-26 11:22:39,560 __main__ [ERROR] This is a test at error level


In [10]:
df = pd.DataFrame.from_records(
    [doc.__dict__ for doc in results.results.documents]
)
df

,rank,doc_id,score,parent,grandparent
0,1,d9e2b7a7-5099-4612-b90a-796f8295ffd5,0.738324,d9e2b7a7-5099-4612-b90a-796f8295ffd5,microphones
1,2,8373d9aa-424a-4cf8-add9-8655f7a6700f,0.732933,8373d9aa-424a-4cf8-add9-8655f7a6700f,microphones
2,3,SR41-392-384-DATASHEET,0.728945,SR41-392-384-DATASHEET,batteries-non-rechargable-primary
3,4,392-384z,0.726830,392-384z,batteries-non-rechargable-primary
4,5,pana-cr1220-ds,0.724038,pana-cr1220-ds,batteries-non-rechargable-primary
5,6,CR2477X-DATASHEET,0.723625,CR2477X-DATASHEET,batteries-non-rechargable-primary
6,7,maxell-alkaline-battery-002-2014,0.721386,maxell-alkaline-battery-002-2014,batteries-non-rechargable-primary
7,8,MU064602-1,0.716878,MU064602-1,microphones
8,9,357-303zb,0.716344,357-303zb,batteries-non-rechargable-primary
9,10,cms0401kl-3x,0.716249,cms0401kl-3x,speakers
